# Epsilon Matrix properties with SequenceOptimizer

This notebook demonstrates using the epsilon properties that optimize towards a matrix instead of a mean value. In this demo you will:
- Create sequences where we optimize towards the self interaction matrix of a sequence of interest.
- Create sequences where we optimize towards a self interaction matrix where we alter the repulsive / attractive values.
- Create sequences where we optimize towards the interaction matrix of a sequence of interest and its target sequence.
- Create sequences where we optimize towards an arbitrary matrix.
- Visualize results

In [ ]:
import goose
from goose.optimize import SequenceOptimizer
import sparrow
from sparrow import Protein
import finches

import numpy as np
import matplotlib.pyplot as plt
import random

#initialize finches forcefields
mf = finches.frontend.mpipi_frontend.Mpipi_frontend()
cf = finches.frontend.calvados_frontend.CALVADOS_frontend()

In [ ]:
# H. sapiens p53 (P04637) N-terminal IDR
p53='MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWFTEDPGPDEAPRMPEAAPPVAPAPAAPTPAAPAPAPSWPLSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENLRKKGEPHHELPPGSTKRALPNNTSSSPQPKKKPLDGEYFTLQIRGRERFEMFRELNEALELKDAQAGKEPGGSRAHSSHLKSKKGQSTSRHKKLMFKTEGPDSD'
p53_idr = p53[1:103]  # N-terminal IDR

# Example 1: Creating a sequence that matches the self-interaction matrix of the N-teriminal IDR of p53

In [ ]:
# Create optimizer
# Note: We set max_iterations to 2000 because this is relatively slow. However,
# if you want something more accurate you can increase this value.
optimizer = goose.SequenceOptimizer(target_length=len(p53_idr), 
                                   max_iterations=2000)

# Disorder constraints
optimizer.add_property(goose.FractionDisorder, 
                      target_value=1, 
                      weight=1)

optimizer.add_property(goose.MatchSelfIntermap,
                      target_sequence=p53_idr)

example1_seq=optimizer.run()

In [ ]:
# function to return the matrix we want to graph. 
def calculate_matrix(seq1, seq2, mod, window_size=15):
    return mod.intermolecular_idr_matrix(seq1, seq2,
                window_size=window_size, disorder_1=False, disorder_2=False)[0][0]

In [ ]:
fig,[a1,a2]=plt.subplots(ncols=2, figsize=[8,5], sharex=True, sharey=True)
vsize=4
ogmat = calculate_matrix(p53_idr, p53_idr, mf)
newmat = calculate_matrix(example1_seq, example1_seq, mf)
im1=a1.imshow(ogmat, cmap='PRGn', vmin=-vsize,vmax=vsize)
im2=a2.imshow(newmat, cmap='PRGn', vmin=-vsize,vmax=vsize)
cbar = fig.colorbar(im1, orientation='horizontal')
cbar = fig.colorbar(im2, orientation='horizontal')
a1.set_title('P53 self interaction')
a2.set_title('Variant self interaction')
plt.show()

# Example 2: Creating a sequence with decreased repuslsive interactions and increased attractive interactions

In [ ]:
# Example 2: Creating a sequence with decreased repuslsive interactions and increased attractive interactions
# based on p53
# Note: We set max_iterations to 3000 because this is relatively slow. However,
# if you want something more accurate you can increase this value.
optimizer = goose.SequenceOptimizer(target_length=len(p53_idr), 
                                   max_iterations=3000)

# Disorder constraints
optimizer.add_property(goose.FractionDisorder, 
                      target_value=1, 
                      weight=1)

# set both sequences to p53_idr because we want to modify the self interaction matrix
# also set homotypic_interaction to True because we are modifying a self interaction matrix. 
# this is necessary here because this class can also be used for heterotypic interactions.
optimizer.add_property(goose.ModifyMatrixValues,
                       sequence=p53_idr,
                       target_sequence=p53_idr,
                       homotypic_interaction=True,
                       repulsive_multiplier=0.5,
                       attractive_multiplier=2)

example2_seq=optimizer.run()

In [ ]:
fig,[a1,a2]=plt.subplots(ncols=2, figsize=[8,5], sharex=True, sharey=True)
vsize=4
ogmat = calculate_matrix(p53_idr, p53_idr, mf)
newmat = calculate_matrix(example2_seq, example2_seq, mf)
im1=a1.imshow(ogmat, cmap='PRGn', vmin=-vsize,vmax=vsize)
im2=a2.imshow(newmat, cmap='PRGn', vmin=-vsize,vmax=vsize)
cbar = fig.colorbar(im1, orientation='horizontal')
cbar = fig.colorbar(im2, orientation='horizontal')
a1.set_title('P53 self interaction')
a2.set_title('Variant self interaction')
plt.show()

# Example 3: Creating a sequence based on the interaction matrix of 2 sequences.

In [ ]:
h1='VGASGSFRLAKSDEPKKSVAFKKTKKEIKKVATPKKASKPKKAASKAPTKKPKATPVKKAKKKLAATPKKAKKPKTVKAKPVKASKPKKAKPVKPKAKSSAKRAGKKK'
prota='MSDAAVDTSSEITTKDLKEKKEVVEEAENGRDAPANGNAENEENGEQEADNEVDEEEEEGGEEEEEEEEGDGEEEDGDEDEEAESATGKRAAEDDEDDDVDTKKQKTDEDD'

**Warning: The following example takes a few minutes to generate the sequence!**

In [ ]:
# Create optimizer
optimizer = goose.SequenceOptimizer(target_length=len(h1), verbose=True,
                                   max_iterations=5000)


# Disorder constraints
optimizer.add_property(goose.FractionDisorder, 
                      target_value=1, 
                      weight=1)

# setting attractive and repulsive multipliers to 1 means we want to match the matrix
optimizer.add_property(goose.ModifyMatrixValues,
                      sequence=h1,
                       target_sequence=prota,
                       attractive_multiplier=1.0,
                       repulsive_multiplier= 1.0,
                       weight=2)


example3_seq=optimizer.run()

In [ ]:
fig,[a1,a2]=plt.subplots(ncols=2, figsize=[8,5], sharex=True, sharey=True)
vsize=15
ogmat = calculate_matrix(h1, prota, mf)
newmat = calculate_matrix(example3_seq, prota, mf)
im1=a1.imshow(ogmat, cmap='PRGn', vmin=-vsize,vmax=vsize)
im2=a2.imshow(newmat, cmap='PRGn', vmin=-vsize,vmax=vsize)
cbar = fig.colorbar(im1, orientation='horizontal')
cbar = fig.colorbar(im2, orientation='horizontal')
a1.set_title('H1/Prota interaction')
a2.set_title('H1(synthetic)/Prota \n interaction')
plt.show()

# Example 4: designing a sequence towards an arbitrary matrix

In [ ]:
# import tool to modify matrix
from goose.backend.optimizer_tools import MatrixManipulation

In [ ]:
test_mat=np.array([[-2,-1,0,1,2],
                     [-1,-1,0,1,1],
                     [0,0,0,0,0],
                     [1,1,0,1,2],
                     [2,1,0,1,2]])
test_mat=MatrixManipulation.scale_matrix_to_size(test_mat, (26,26))

In [ ]:
# Create optimizer
optimizer = goose.SequenceOptimizer(target_length=50,
                                   max_iterations=5000)


optimizer.add_property(goose.MatchArbitraryMatrix,
                       arbitrary_matrix=test_mat,
                       weight=5)


example4_seq=optimizer.run()

In [ ]:
fig,[a1,a2]=plt.subplots(ncols=2, figsize=[8,5])#, sharex=True, sharey=True)
vsize=4
ogmat = test_mat
newmat = calculate_matrix(example4_seq, example4_seq, mf)
im1=a1.imshow(ogmat, cmap='PRGn', vmin=-vsize,vmax=vsize)
im2=a2.imshow(newmat, cmap='PRGn', vmin=-vsize,vmax=vsize)
cbar = fig.colorbar(im1, orientation='horizontal')
cbar = fig.colorbar(im2, orientation='horizontal')
a1.set_title('arbitrary matrix')
a2.set_title('generated seq')
plt.show()